In [0]:
from pyspark.sql import DataFrameWriter
import pandas as pd
from pyspark.sql.functions import *

In [0]:
dbutils.secrets.listScopes()

[SecretScope(name='scope31dec')]

In [0]:
dbutils.secrets.list(scope = 'scope31dec')

[SecretMetadata(key='blobkeysecret31dec')]

In [0]:
spark.conf.set("fs.azure.account.key.healthcareblob31dec.dfs.core.windows.net",
    dbutils.secrets.get(scope="scope31dec", key="blobkeysecret31dec"))

In [0]:
display(dbutils.fs.ls("abfss://rawdata@healthcareblob31dec.dfs.core.windows.net"))

path,name,size,modificationTime
abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/Patient_records.csv,Patient_records.csv,5110,1704010961000
abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/disease.csv,disease.csv,1489,1704010960000
abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/group.csv,group.csv,4390,1704010960000
abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/hospital.csv,hospital.csv,1328,1704010960000
abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/subgroup.csv,subgroup.csv,561,1704010960000
abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/subscriber.csv,subscriber.csv,12061,1704010960000


In [0]:
disease_data = spark.read.csv("abfss://rawdata@healthcareblob31dec.dfs.core.windows.net/disease.csv", header = True, inferSchema= True)

In [0]:
disease_data.show(10, False)

+---------+----------+------------+
|subgrp_id|disease_id|disease_name|
+---------+----------+------------+
|S101     |110001    |Beriberi    |
|S101     |110002    |Scurvy      |
|S101     |110003    |Goitre      |
|S101     |110004    |Osteoporosis|
|S101     |110005    |Rickets     |
|S101     |110006    |Anaemia     |
|S102     |110007    |Fractures   |
|S102     |110008    |Heart Attack|
|S102     |110009    |Burns       |
|S102     |110010    |Choking     |
+---------+----------+------------+
only showing top 10 rows



In [0]:
disease_data.printSchema()

root
 |-- subgrp_id: string (nullable = true)
 |-- disease_id: integer (nullable = true)
 |-- disease_name: string (nullable = true)



In [0]:
output_container_path = "abfss://stagingdata@healthcareblob31dec.dfs.core.windows.net"
output_blob_folder = "stagingdata/"
disease_data.coalesce(1).write.mode("overwrite").option("header" , "true"). format("com.databricks.spark.csv").save(output_blob_folder)
files = dbutils.fs.ls(output_blob_folder)
outputfile = [x for x in files if x.name.startswith("part-")]
dbutils.fs.mv(outputfile[0].path, "%s/disease_data_stage.csv"% output_container_path)

True